In [1]:
# GENETIC ALGORITHM for solving linear regression:
# we want to solve: y = a(x1) + b(x2) + c(x3) + d --> find best value for paramter a,b,c,d
#
# Source:https://medium.com/the-andela-way/on-genetic-algorithms-and-their-application-in-solving-regression-problems-4e37ac1115d5
# https://medium.com/the-andela-way/on-genetic-algorithms-and-their-application-in-solving-regression-problems-4e37ac1115d5
# # rewritten by D.Adytia (didit.adytia@gmail.com) ver. 2021-04-20

from random import random, sample, choice
from math import floor
from tqdm import tqdm                  # instantly make your loops show a smart progress meter
from numpy import array, dot, mean
from numpy.linalg import pinv
from sys import exit

In [2]:
def generate_data():
    """
    We will generate data with a clear pattern.
    This ensures we have an idea of the desired result.
    This is only for demonstration purposes, real data is needed in practice.
    """
    coeff = [0.4, -0.3, 0.2, -0.1]
    x = [[random() for j in range(len(coeff))] for i in range(1000)]   # generate random data set with range of 1000
    y = [dot(i, coeff) for i in x]
    print('Input:', x)
    print('Output:', y)
    return array(x), array(y)


In [3]:
def multiple_linear_regression(inputs, outputs):
    """
    Get the best expected outcome.
    This is expected to equal the coefficients in generate_data().
    """
    X, Y = array(inputs), array(outputs)
    X_t, Y_t = X.transpose(), Y.transpose()
    coeff = dot((pinv((dot(X_t, X)))), (dot(X_t, Y)))
    Y_p = dot(X, coeff)
    Y_mean = mean(Y)
    SST = array([(i - Y_mean) ** 2 for i in Y]).sum()         # SST: the total error in a model, it is the sum of all deviations squared
    SSR = array([(i - j) ** 2 for i, j in zip(Y, Y_p)]).sum() # SSR: a measure of the explained variation in SST.
    COD = (1 - (SSR / SST)) * 100.0                           # COD: stands for ‘coefficient of determination’ which is basically a measure of how good a model is.
    av_error = (SSR / len(Y))
    return {'COD': COD, 'coeff': coeff, 'error': av_error}    # error: the average error, is an average of all deviations from expected values

In [4]:
def check_termination_condition(best_individual):
    """
    Check if the current_best_individual is better of equal to the expected.
    """
    if ((best_individual['COD'] >= 99.5)
            or (generation_count == max_generations)):
        return True
    else:
        return False

In [5]:
def create_individual(individual_size):
    """
    Create an individual.
    """
    return [random() for i in range(individual_size)]   # To create an initial individual, we will use random assigning of variables.


def create_population(individual_size, population_size):
    """
    Create an initial population.
    """
    return [create_individual(individual_size) for i in range(population_size)]


def get_fitness(individual, inputs):
    """
    Calculate the fitness of an individual.
    Return the Coefficient of Determination, average error and weight.
    We use the error to get the best individual.
    """
    predicted_outputs = dot(array(inputs), array(individual))
    output_mean = mean(outputs)
    SST = array(
        [(i - output_mean) ** 2 for i in outputs]).sum()
    SSR = array(
        [(i - j) ** 2 for i, j in zip(outputs, predicted_outputs)]).sum()
    COD = (1 - (SSR / SST)) * 100.0
    av_error = (SSR / len(outputs))
    return {'COD': COD, 'error': av_error, 'coeff': individual}


def evaluate_population(population):
    """
    Evaluate a population of individuals and return the best among them.
    """
    fitness_list = [get_fitness(individual, inputs)
                    for individual in tqdm(population)]
    error_list = sorted(fitness_list, key=lambda i: i['error'])
    best_individuals = error_list[: selection_size]
    best_individuals_stash.append(best_individuals[0]['coeff'])
    print('Error: ', best_individuals[0]['error'],
          'COD: ', best_individuals[0]['COD'])
    return best_individuals


def crossover(parent_1, parent_2):
    """
    Return offspring given two parents.
    Unlike real scenarios, genes in the chromosomes aren't necessarily linked.
    """
    child = {}
    loci = [i for i in range(0, individual_size)]
    loci_1 = sample(loci, floor(0.5*(individual_size)))
    loci_2 = [i for i in loci if i not in loci_1]
    chromosome_1 = [[i, parent_1['coeff'][i]] for i in loci_1]
    chromosome_2 = [[i, parent_2['coeff'][i]] for i in loci_2]
    child.update({key: value for (key, value) in chromosome_1})
    child.update({key: value for (key, value) in chromosome_2})
    return [child[i] for i in loci]


def mutate(individual):
    """
    Mutate an individual.
    The gene transform decides whether we'll add or deduct a random value.
    """
    loci = [i for i in range(0, individual_size)]
    no_of_genes_mutated = floor(probability_of_gene_mutating*individual_size)
    loci_to_mutate = sample(loci, no_of_genes_mutated)
    for locus in loci_to_mutate:
        gene_transform = choice([-1, 1])
        change = gene_transform*random()
        individual[locus] = individual[locus] + change
    return individual


def get_new_generation(selected_individuals):
    """
    Given selected individuals, create a new population by mating them.
    Here we also apply variation operations like mutation and crossover.
    """
    parent_pairs = [sample(selected_individuals, 2)
                    for i in range(population_size)]
    offspring = [crossover(pair[0], pair[1]) for pair in parent_pairs]
    offspring_indices = [i for i in range(population_size)]
    offspring_to_mutate = sample(
        offspring_indices,
        floor(probability_of_individual_mutating*population_size)
    )
    mutated_offspring = [[i, mutate(offspring[i])]
                         for i in offspring_to_mutate]
    for child in mutated_offspring:
        offspring[child[0]] = child[1]
    return offspring

In [6]:
# MAIN CODE
# Initialization
populations = [500, 1000, 1500, 2000]
for pop_size in populations:
  print('===> Population Size:', pop_size)
  inputs, outputs = generate_data()
  individual_size = len(inputs[0])
  print('Individual Size:', individual_size)
  population_size = pop_size
  selection_size  = floor(0.1*population_size)
  max_generations = 50
  probability_of_individual_mutating = 0.1
  probability_of_gene_mutating = 0.25
  best_possible = multiple_linear_regression(inputs, outputs)
  best_individuals_stash = [create_individual(individual_size)]
  initial_population = create_population(individual_size, 1000)
  current_population = initial_population
  termination = False
  generation_count = 0
  while termination is False:
      current_best_individual = get_fitness(best_individuals_stash[-1], inputs)
      print('Generation: ', generation_count)
      best_individuals   = evaluate_population(current_population)
      current_population = get_new_generation(best_individuals)
      termination        = check_termination_condition(current_best_individual)
      generation_count  += 1
  else:
    print()
    print('===> Population Size:', pop_size)
    print(get_fitness(best_individuals_stash[-1], inputs))

===> Population Size: 500

Input: [[0.944515204290901, 0.4763436804110116, 0.6405520179119861, 0.9212594723077915], [0.18141946307847234, 0.3609046863057114, 0.9117238527499122, 0.39777821166547], [0.25824331058992633, 0.14199948545167373, 0.014987987310297712, 0.28127056367139625], [0.6015293344466195, 0.14411834925018696, 0.8135832164991672, 0.9983989262578218], [0.7091518542248884, 0.8212754955889405, 0.6960678120902212, 0.6895253635320181], [0.7638667327984908, 0.6565447352632783, 0.2757968395442051, 0.8120280434603951], [0.4027400649399293, 0.9449127978964297, 0.6699178022131976, 0.04289662636795555], [0.5296174212608076, 0.8945631495778703, 0.4229001071229278, 0.5892461388659579], [0.9930372849232425, 0.5122516192667199, 0.7112147873778817, 0.8430749817577781], [0.5621561125953498, 0.2942494348375091, 0.21745737405568588, 0.3236125517850391], [0.9257466560296675, 0.7940124063089682, 0.14792878447755398, 0.18435834980952004], [0.6851312216716725, 0.17541853682404407, 0.18629233433

100%|█████████████████████████████████████| 1000/1000 [00:00<00:00, 1495.07it/s]


Error:  0.029097781894295578 COD:  -16.605933787060636
Generation:  1


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 1486.52it/s]


Error:  0.018147316182109005 COD:  27.27676780206052
Generation:  2


...

Error:  0.00023571246046406657 COD:  99.1021843074786
Generation:  10


100%|█████████████████████████████████████| 2000/2000 [00:01<00:00, 1497.29it/s]


Error:  0.00011885290917144067 COD:  99.54729585892127
Generation:  11


100%|█████████████████████████████████████| 2000/2000 [00:01<00:00, 1497.05it/s]

Error:  0.00011885290917144067 COD:  99.54729585892127

===> Population Size: 2000
{'COD': 99.54729585892127, 'error': 0.00011885290917144067, 'coeff': [0.405311336775528, -0.33209691401439456, 0.20826520188205244, -0.084664661799434]}
